* [#1](https://github.com/salgo60/Naturreservat-Sverige/issues/1) 
* denna Notebook [syncWDOSM](https://github.com/salgo60/Naturreservat-Sverige/blob/main/Notebook/syncWDOSM.ipynb)



In [1]:

import time
from datetime import datetime 
import glob
import pandas as pd
from datetime import date

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))



Start: 2026-02-11 14:35:14


In [ ]:
import requests
import json
import time
import random
import os
from SPARQLWrapper import SPARQLWrapper, JSON

# -------------------------
# CONFIG (snäll!)
# -------------------------
BATCH_SIZE = 10
BASE_SLEEP = 30  # sek mellan batchar

OVERPASS_SERVERS = [
    "https://overpass-api.de/api/interpreter",
    "https://lz4.overpass-api.de/api/interpreter",
    "https://z.overpass-api.de/api/interpreter"
]

CACHE_DIR = "overpass_cache"
os.makedirs(CACHE_DIR, exist_ok=True)

# -------------------------
# 1. HÄMTA WIKIDATA
# -------------------------
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setReturnFormat(JSON)

query = """
SELECT ?item WHERE {
  ?item wdt:P31 wd:Q179049;
        wdt:P17 wd:Q34.
  OPTIONAL {?item wdt:P402 ?r}
  OPTIONAL {?item wdt:P10689 ?w}
  FILTER(!BOUND(?r) && !BOUND(?w))
}
"""

sparql.setQuery(query)
res = sparql.query().convert()

qids = [
    b["item"]["value"].split("/")[-1]
    for b in res["results"]["bindings"]
]

print("Totala QID:", len(qids))


# -------------------------
# Helpers
# -------------------------
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n], i//n


def overpass_query(qids):
    regex = "|".join(qids)

    return f"""
    [out:json][timeout:120];
    (
      node["wikidata"~"^({regex})$"];
      way["wikidata"~"^({regex})$"];
      relation["wikidata"~"^({regex})$"];
    );
    out tags center;
    """


def polite_request(query, retries=5):
    for attempt in range(retries):

        server = random.choice(OVERPASS_SERVERS)

        try:
            r = requests.post(server, data=query, timeout=180)

            if r.status_code == 200:
                return r.json()

            if r.status_code in (429, 504):
                wait = (2 ** attempt) * 30
                print(f"Server stress → väntar {wait}s")
                time.sleep(wait)
                continue

        except Exception as e:
            wait = (2 ** attempt) * 30
            print(f"Fel {e} → väntar {wait}s")
            time.sleep(wait)

    return None

# -------------------------
# RESUME LOGIC
# -------------------------
existing = []
for f in os.listdir(CACHE_DIR):
    if f.startswith("batch_") and f.endswith(".json"):
        try:
            n = int(f.split("_")[1].split(".")[0])
            existing.append(n)
        except:
            pass

start_batch = max(existing) + 1 if existing else 0
print("Resumerar från batch:", start_batch)


# -------------------------
# 2. Kör batchar
# -------------------------
for batch, idx in chunks(qids, BATCH_SIZE):

    if idx < start_batch:
        continue

    cache_file = f"{CACHE_DIR}/batch_{idx}.json"

    if os.path.exists(cache_file):
        print(f"Batch {idx} redan klar, hoppar över")
        continue

    print(f"\nBatch {idx} ({len(batch)} QID)")

    q = overpass_query(batch)
    result = polite_request(q)

    if result:
        with open(cache_file, "w") as f:
            json.dump(result, f)

        hits = len(result.get("elements", []))
        print(f"Träffar: {hits}")

    sleep_time = BASE_SLEEP + random.randint(0,15)
    print(f"Sover {sleep_time}s...")
    time.sleep(sleep_time)

print("\nKLART 🎉")


Totala QID: 2582
Resumerar från batch: 43

Batch 43 (10 QID)
Server stress → väntar 30s
Server stress → väntar 60s
Server stress → väntar 120s
Träffar: 7
Sover 39s...

Batch 44 (10 QID)
Träffar: 6
Sover 35s...

Batch 45 (10 QID)
Server stress → väntar 30s
Server stress → väntar 60s
Server stress → väntar 120s
Server stress → väntar 240s
Träffar: 4
Sover 37s...

Batch 46 (10 QID)
Träffar: 3
Sover 36s...

Batch 47 (10 QID)
Träffar: 6
Sover 43s...

Batch 48 (10 QID)
Träffar: 3
Sover 32s...

Batch 49 (10 QID)
Träffar: 6
Sover 37s...

Batch 50 (10 QID)
Server stress → väntar 30s
Träffar: 6
Sover 32s...

Batch 51 (10 QID)
Träffar: 5
Sover 38s...

Batch 52 (10 QID)
Träffar: 6
Sover 36s...

Batch 53 (10 QID)
Server stress → väntar 30s
Träffar: 5
Sover 45s...

Batch 54 (10 QID)
Träffar: 2
Sover 36s...

Batch 55 (10 QID)
Träffar: 6
Sover 32s...

Batch 56 (10 QID)
Server stress → väntar 30s
Träffar: 8
Sover 45s...

Batch 57 (10 QID)
Träffar: 5
Sover 42s...

Batch 58 (10 QID)
Träffar: 5
Sover 34s.

In [ ]:
qs_lines = []
CACHE_DIR = "overpass_cache"
for f in os.listdir(CACHE_DIR):
    if not f.endswith(".json"):
        continue

    with open(os.path.join(CACHE_DIR, f)) as fh:
        data = json.load(fh)

    for el in data.get("elements", []):
        tags = el.get("tags", {})
        qid = tags.get("wikidata")

        if not qid:
            continue

        if el["type"] == "relation":
            qs_lines.append(f'{qid}\tP402\t"{el["id"]}"')

        elif el["type"] == "way":
            qs_lines.append(f'{qid}\tP10689\t"{el["id"]}"')

# spara
with open("quickstatements.txt", "w") as f:
    f.write("\n".join(qs_lines))

print("KLART – quickstatements.txt skapad")


In [ ]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))